In [1]:
import numpy as np
import pickle
import math

In [2]:
road_network = np.array(np.load("roads/road"))
print(road_network[:5])
print("Size of road network is : " , road_network.shape)

[[0 3 6 0 0 8 0 0 0 0]
 [8 0 0 0 0 0 0 0 9 2]
 [0 7 0 0 0 0 8 0 0 0]
 [0 0 0 0 0 0 0 4 7 0]
 [0 9 0 0 0 0 3 0 0 0]]
Size of road network is :  (10, 10)


In [3]:
vehicle = np.array(np.load("roads/vehicle"))
print(vehicle[:5])
print("Size of vechile is : " , vehicle.shape )

[[1 0 5 3 7]
 [7 8 7 8 7]
 [3 7 0 1 9]
 [6 0 5 2 6]
 [8 7 0 1 9]]
Size of vechile is :  (100, 5)


In [4]:
time = np.array(np.load("roads/time", encoding='bytes'))
print(time[:5])
print("Size of time is : " , time.shape)

[[1.04966149]
 [2.30189821]
 [2.64456047]
 [2.72793688]
 [3.57308425]]
Size of time is :  (100, 1)


In [5]:
class environment :
    def __init__ (self , road_network):
        self.road = road_network
        self.status_of_road = np.zeros(road_network.shape)
        
    def determine_speed(self , x):
        x = 2
        return math.exp(0.5 * x) / (1 + math.exp(0.5*x)) + 15 / (1 + math.exp(0.5 * x))
    

In [6]:
class agents :
    
    #these are the variables
    detailed_status = np.array( (0 , 0) )         #this is to store the timing of each of the vehicle when it crossed the given node
    path = np.array((0 , 0))
    time_status = np.array((0 , 0))
    done = np.array((0 , 0))
    
    def __init__ (self, time , vehicle):
        self.path = vehicle
        self.time_status = time
        self.detailed_status = np.hstack ([time , math.inf * np.ones((vehicle.shape[0] , vehicle.shape[1] - 1)) ])
        
        #To keep track of which point the vehicle is in the path
        self.done =  np.hstack ([np.ones(time.shape , dtype=  bool) , np.zeros((vehicle.shape[0] , vehicle.shape[1] - 1), dtype= bool) ])
        
    def which_vehicle(self):
        return self.time_status.argmin()                 #this returns the index of the minimum element
        

In [7]:
#initialising the agent and the environment

agent = agents(time , vehicle)
env = environment (road_network)

In [8]:
while(not np.all(agent.done)):
    
    current = agent.which_vehicle()
    index = -1
    #print("Current vehicle : " , current )
    
    for temp in range(agent.path.shape[1]):
        if (agent.done[current, temp] == False):
            index= temp;
            break

    if (index == -1):
        env.status_of_road[agent.path[current ,3] , agent.path[current , 4]] -=1
        agent.time_status[current , 0] = math.inf 
        continue
    
    agent.done[current, index] = True
    #print(agent.done)
   
    x = agent.path[current , index-1]
    y = agent.path[current , index]
    
    number_of_vehicle = env.status_of_road[x , y]
    env.status_of_road[x , y] += 1
    #print(x , y ,index , number_of_vehicle)
    #this means that the road is free
    if index-2 >= 0:
        env.status_of_road[agent.path[current , index-2] ,x] -=1
    #print(number_of_vehicle)
    speed = env.determine_speed(number_of_vehicle)
    time_required = env.road[x , y] / speed
    time_required = time_required * 60
    
    print(agent.time_status[current , 0] , time_required)
    agent.time_status[current , 0] += time_required
    
    agent.detailed_status[current,  index] = agent.time_status[current , 0]
    

1.0496614869703016 100.73071954379016
2.301898206346439 62.956699714868854
2.6445604682538404 50.36535977189508
2.7279368773584745 88.1393796008164
3.5730842454515277 12.59133994297377
4.291011648347961 37.77401982892131
4.691794758090987 12.59133994297377
5.571473171394375 25.18267988594754
6.581644815388371 88.1393796008164
6.590895568648802 25.18267988594754
7.086766254692237 62.956699714868854
7.430416543226079 113.32205948676393
7.533138006235388 113.32205948676393
7.821580230851338 88.1393796008164
7.852037592150623 37.77401982892131
8.75430812780004 25.18267988594754
9.180015276611556 113.32205948676393
9.423906066737333 62.956699714868854
9.671091829206132 113.32205948676393
10.098378901716849 100.73071954379016
10.250563542946546 88.1393796008164
11.095733105919994 100.73071954379016
15.883991672658706 100.73071954379016
16.164424188425297 75.54803965784262
16.619693671051657 113.32205948676393
17.283134701064757 62.956699714868854
17.880965450827432 37.77401982892131
18.99776

In [9]:
print(agent.detailed_status)

[[  1.04966149 101.78038103 202.51110057 315.83316006 366.19851983]
 [  2.30189821  65.25859792  77.84993786 140.80663758 153.39797752]
 [  2.64456047  53.00992024 128.5579599  166.33197973 191.51465961]
 [  2.72793688  90.86731648 191.59803602 216.78071591 317.51143545]
 [  3.57308425  16.16442419  91.71246385 129.48648368 154.66916356]
 [  4.29101165  42.06503148 117.61307114 155.38709096 180.56977085]
 [  4.69179476  17.2831347   80.23983442  92.83117436 168.37921402]
 [  5.57147317  30.75415306  93.71085277 207.03291226 320.35497175]
 [  6.58164482  94.72102442 132.49504425 245.81710373 258.40844367]
 [  6.59089557  31.77357545 145.09563494 157.68697488 271.00903437]
 [  7.08676625  70.04346597 107.8174858  221.13954529 246.32222517]
 [  7.43041654 120.75247603 234.07453552 322.21391512 359.98793495]
 [  7.53313801 120.85519749 208.99457709 221.58591704 297.13395669]
 [  7.82158023  95.96095983 133.73497966 158.91765955 221.87435926]
 [  7.85203759  45.62605742  70.80873731 133.765

In [10]:
print(agent.time_status)

[[         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [398.5315654 ]
 [         inf]
 [413.44312577]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [         inf]
 [      

In [11]:
print(env.status_of_road)

[[0. 0. 2. 0. 0. 1. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
